In [ ]:
import os
import io
import time
import logging
import random
import re
import sqlite3
from datetime import datetime, timedelta
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe
from sqlalchemy import create_engine, inspect
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.errors import HttpError
from gspread_formatting import CellFormat, TextFormat, Color, Borders, Border, set_column_width, format_cell_range
from gspread_formatting import batch_updater
from google.auth.transport.requests import Request
from googleapiclient.http import HttpRequest
import ipywidgets as widgets
import tkinter as tk
from IPython.display import display, clear_output
from google_auth_oauthlib.flow import InstalledAppFlow
from tkinter import messagebox
#from google.oauth2.credentials import Credentials as UserCredentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

<strong>LESSON LEARNED</strong>

each time code updated or changes made 
<strong>START OVER</strong>
duplicate file and update name

# SETUP DIRECTORIES

In [ ]:
script_dir = os.getcwd()
print(script_dir)

#background_data_dir = script_dir + '\\' + 'Background Data Folder'
"""os.path.join adds the appropriate path separator (\ or /) based on the operating system"""
background_data_dir = os.path.join(script_dir, 'Background Data Folder')
if not os.path.exists(background_data_dir):
    os.makedirs(background_data_dir)

#report_dir = script_dir + '\\' + 'Reports Folder'
reports_dir = os.path.join(script_dir, 'Reports Folder') 
if not os.path.exists(reports_dir):
    os.makedirs(reports_dir)

data_supplied_dir = os.path.join(script_dir, 'Data Supplied Folder')
if not os.path.exists(data_supplied_dir):
    os.makedirs(data_supplied_dir)

c:\Users\nicola\Desktop\VisualCode Workspace\Team Meeting Update

# DETERMINE REPORTING DATE
display as dropdown 

In [ ]:
MasterData_db_path = os.path.join(background_data_dir, 'master_data.db')
conn = sqlite3.connect(MasterData_db_path) #f"{background_data_dir}/master_data.db"
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT ReportingDate FROM MasterData")   
periods = [row[0] for row in cursor.fetchall()]
conn.close()

out = widgets.Output()

dropdown = widgets.Dropdown(
    options=periods,
    description='Reporting Date:',
    disabled=False,
)


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global selected_reporting_date
        selected_reporting_date = change['new']
        with out:
            clear_output()
            print(f"Selected Period: {selected_reporting_date}")

dropdown.observe(on_change)


In [ ]:
with out:
    display(dropdown)

display(out)

In [ ]:
selected_reporting_date # just a check

'02 May 2025'

In [ ]:
str_ReportingDate = selected_reporting_date
datetime_ReportingDate = datetime.strptime(selected_reporting_date, '%d %b %Y').date()
formatted_ReportingDate = datetime_ReportingDate.strftime('%d %m %Y')

RunDate = datetime_ReportingDate - timedelta(days=1)
formatted_RunDate = f"{RunDate .day:02d} {RunDate .month:02d} {RunDate .year}"
#formatted_RunDate = RunDate.strftime('%d %m %Y')

#Designated_Filename = 'KIT 3 Online Reporting {}.xlsx'.format(formatted_RunDate)
Designated_Filename = f"KIT 3 Online Reporting Data {formatted_RunDate}.xlsx"

1. Using f-string with manual formatting: 

    formatted_RunDate = f"{RunDate.day:02d} {RunDate.month:02d} {RunDate.year}"

        RunDate.day:02d formats the day with two digits (e.g., 01, 09, 23).

        RunDate.month:02d does the same for the month.

        RunDate.year just inserts the full year (e.g., 2025).

    This approach is explicit and lets you control each component individually.

2. Using .strftime()

    formatted_RunDate = RunDate.strftime('%d %m %Y')


        %d formats the day as a two-digit number.

        %m formats the month as a two-digit number.

        %Y formats the year as a four-digit number.

    This method is cleaner and more concise, especially useful when dealing with more complex date formats.


1. Using .format()

    Designated_Filename = 'KIT 3 Online Reporting {}.xlsx'.format(formatted_RunDate)

    This uses Python's older .format() method to insert the value of formatted_RunDate into the {} placeholder.

    If you had more placeholders, you could insert multiple values like .format(a, b, c).

2. Using an f-string

    Designated_Filename = f"KIT 3 Online Reporting Data {formatted_RunDate}.xlsx"

    This uses a formatted string literal (f-string), available from Python 3.6+.

    You embed variables directly in the string with {}.

    It’s generally cleaner and more readable, especially when combining multiple variables or expressions.

## ESTABLISH CORRESPONDING DATA

In [ ]:
conn = sqlite3.connect(MasterData_db_path)
cursor = conn.cursor()

# Query the row where ReportingDate matches
cursor.execute("SELECT * FROM MasterData WHERE ReportingDate = ?", (str_ReportingDate,))
row = cursor.fetchone()

if row:
    column_names = [description[0] for description in cursor.description]
    
    for key, value in zip(column_names, row):
        if key in ["StartDate", "EndDate", "ReportingDate"] and isinstance(value, str):
            # Convert string to datetime.date
            try:
                # For StartDate and EndDate, use ISO format (YYYY-MM-DD)
                if key in ["StartDate", "EndDate"]:
                    value = datetime.strptime(value, "%Y-%m-%d").date()
                else:
                    # ReportingDate is in 'dd MMM YYYY' format
                    value = datetime.strptime(value, "%d %b %Y").date()
            except ValueError:
                value = None  # Handle parsing error gracefully
        
        # Store the value as a global variable (if needed)
        globals()[key] = value
        
        # Print out the key, value, and its type
        print(f"{key} = {value} ({type(value).__name__})")
else:
    print(f"No row found for ReportingDate = {formatted_ReportingDate}")

# Close the database connection
conn.close()

id = 17 (int)
StartDate = 2025-04-24 (date)
EndDate = 2025-05-01 (date)
WeekNumber = 17 (int)
YearWeek = 202517 (int)
CurrentSheetName = 24 Apr - 30 Apr (str)
CurrentPeriod = 24 Apr 2025 - 01 May 2025 (str)
PeriodName = 202517 - 24 Apr 2025 - 01 May 2025 (str)
PreviousSheetName = 17 Apr - 23 Apr (str)
NextSheetName = 01 May - 07 May (str)
ReportingDate = 2025-05-02 (date)

In [ ]:
# check execution 
YeakWeek = YearWeek
WeekNumber = WeekNumber
CurrentSheetName = CurrentSheetName
CurrentPeriod = CurrentPeriod
PeriodName = PeriodName
PreviousSheetName = PreviousSheetName
NextSheetName = NextSheetName
ReportingDate = ReportingDate

# CHECK GOOGLE DRIVE and GMAIL CONNECTION

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Print script directory and its contents for debugging
logging.info(f"Notebook directory: {background_data_dir}")
logging.info(f"Directory contents: {os.listdir(background_data_dir)}")

# Define the service account file path
GOOGLE_SERVICE_ACCOUNT_FILE = os.getenv(
    'SERVICE_ACCOUNT_FILE',
    os.path.join(background_data_dir, 'GoogleAuth.json')
)

GMAIL_SERVICE_ACCOUNT_FILE = os.getenv(
    'SERVICE_ACCOUNT_FILE',
    os.path.join(background_data_dir, 'GmailAuth.json')
)

# Print the path being used
logging.info(f"Attempting to use service account file: {GOOGLE_SERVICE_ACCOUNT_FILE}")
logging.info(f"Attempting to use service account file: {GMAIL_SERVICE_ACCOUNT_FILE}")

# Check if the file exists
if not os.path.exists(GOOGLE_SERVICE_ACCOUNT_FILE):
    logging.error(f"Service account file not found: {GOOGLE_SERVICE_ACCOUNT_FILE}")
    raise FileNotFoundError(f"Service account file not found: {GOOGLE_SERVICE_ACCOUNT_FILE}")

logging.info("File found, proceeding with authentication...")
# Add your Google Sheets authentication code here

if not os.path.exists(GMAIL_SERVICE_ACCOUNT_FILE):
    logging.error(f"Service account file not found: {GMAIL_SERVICE_ACCOUNT_FILE}")
    raise FileNotFoundError(f"Service account file not found: {GMAIL_SERVICE_ACCOUNT_FILE}")

logging.info("File found, proceeding with authentication...")
# Add your Google Sheets authentication code here

def authenticate_gdrive():
    #SCOPES = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
    SCOPES = ['https://www.googleapis.com/auth/drive']
    try:
        creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        gc = gspread.authorize(creds)
        drive_service = build('drive', 'v3', credentials=creds)
        return gc, drive_service
    except Exception as e:
        logging.error(f"Failed to authenticate with Google API: {e}")
        raise


def authenticate_gsheets():
    """Authenticate and return gspread client and Google Sheets API service."""
    # For gspread
    #SCOPES = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    #creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', SCOPES)
    #gc = gspread.authorize(creds)
    
    # For Google Sheets API
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    try:
        creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        gc = gspread.authorize(creds)
        sheets_service = build('sheets', 'v4', credentials=creds)
        return gc, sheets_service
    except Exception as e:
        logging.error(f"Failed to authenticate with Google Sheets: {e}")


def download_file(file_id: str, output_filename: str):
    try:
        _, drive_service = authenticate_gdrive()
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        )

        with io.FileIO(output_filename, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                logging.info(f"Download {int(status.progress() * 100)}% complete.")

        logging.info(f"File downloaded as {output_filename}")
    except Exception as e:
        logging.error(f"Failed to download file: {e}")
        raise

def execute_with_backoff(func,*args,max_retries=5,**kwargs):
    for attempt in range(max_retries):
        try:
            result=func(*args,**kwargs)
            if isinstance(result,HttpRequest):
                result=result.execute()
            return result
        except(HttpError,gspread.exceptions.APIError) as e:
            if 'Quota exceeded' in str(e) or getattr(e,'status',0)==429:
                if attempt==max_retries-1:
                    logging.error(f"Max retries reached for {func.__name__}: {e}")
                    raise
                sleep_time=(2**attempt)+(random.randint(0,1000)/1000)
                logging.warning(f"Quota exceeded, retrying in {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)
            else:
                raise
        except Exception as e:
            logging.error(f"Unexpected error in {func.__name__}: {e}")
            raise


def add_or_overwrite_sheet(spreadsheet_id: str, sheet_name: str, template_sheet_id: int = None, data: list = None):
    try:
        # Authenticate
        gc, sheets_service = authenticate_gsheets()
        
        # Open the spreadsheet
        spreadsheet = execute_with_backoff(gc.open_by_key, spreadsheet_id)
        
        # Get current worksheets
        worksheets = execute_with_backoff(spreadsheet.worksheets)
        worksheet_titles = [sheet.title for sheet in worksheets]
        worksheet_ids = {sheet.title: sheet.id for sheet in worksheets}

        # Check if the sheet already exists
        if sheet_name in worksheet_titles:
            logging.info(f"Sheet '{sheet_name}' already exists. Deleting it.")
            sheet_id = worksheet_ids[sheet_name]
            delete_request = {
                'requests': [{
                    'deleteSheet': {
                        'sheetId': sheet_id
                    }
                }]
            }
            execute_with_backoff(
                sheets_service.spreadsheets().batchUpdate,
                spreadsheetId=spreadsheet_id,
                body=delete_request
            )
            logging.info(f"Deleted existing sheet '{sheet_name}'.")
            time.sleep(1)  # Brief delay to ensure deletion is processed

        # Create a new sheet by duplicating a template (if provided) or adding a blank sheet
        if template_sheet_id:
            new_sheet = execute_with_backoff(
                spreadsheet.duplicate_sheet,
                source_sheet_id=template_sheet_id,
                new_sheet_name=sheet_name
            )
            logging.info(f"Created new sheet '{sheet_name}' by duplicating template.")
        else:
            new_sheet = execute_with_backoff(
                spreadsheet.add_worksheet,
                title=sheet_name,
                rows=100,
                cols=20
            )
            logging.info(f"Created new blank sheet '{sheet_name}'.")

        # If data is provided, update the sheet with new data
        if data:
            new_sheet = execute_with_backoff(spreadsheet.worksheet, sheet_name)
            execute_with_backoff(
                new_sheet.update,
                values=data,
                range_name='A1'
            )
            logging.info(f"Updated sheet '{sheet_name}' with new data.")

        return new_sheet, worksheet_ids

    except Exception as e:
        logging.error(f"Failed to add or overwrite sheet '{sheet_name}': {e}")
        raise

2025-05-21 15:34:34,139 - INFO - Notebook directory: c:\Users\nicola\Desktop\VisualCode Workspace\Team Meeting Update\Background Data Folder
2025-05-21 15:34:34,142 - INFO - Directory contents: ['Construct_MasterData.ipynb', 'Construct_Previous.ipynb', 'Construct_WeekDates.ipynb', 'error_log.txt', 'GmailAuth.json', 'GmailToken.json', 'GoogleAuth.json', 'master_data.db', 'previous_errors_n_reponse_times_data.db', 'Previous_Error_n_ResponseTimes202516.sql', 'Previous_Error_n_ResponseTimes202517.sql', 'week_dates.db']
2025-05-21 15:34:34,143 - INFO - Attempting to use service account file: c:\Users\nicola\Desktop\VisualCode Workspace\Team Meeting Update\Background Data Folder\GoogleAuth.json
2025-05-21 15:34:34,144 - INFO - Attempting to use service account file: c:\Users\nicola\Desktop\VisualCode Workspace\Team Meeting Update\Background Data Folder\GmailAuth.json
2025-05-21 15:34:34,146 - INFO - File found, proceeding with authentication...
2025-05-21 15:34:34,148 - INFO - File found, proceeding with authentication...

## standardize client names

In [ ]:
def clean_string(s):
    return str(s).strip().lower()


def extract_dates(text):
    match = re.search(r'(\d{1,2} \w{3}) \d{4} - (\d{1,2} \w{3}) \d{4}', text)
    if match:
        return f"{match.group(1)} - {match.group(2)}"
    return None  # or return text if no match is found


In [ ]:
def clients_df_to_custom_dict(df):
    result = {}
    for _, row in df.iterrows():
        # Initialize list with 'Other' value (assuming it's always non-null)
        values = [row['Other1']]
        # Append non-null values from Other2 and Other3
        for col in ['Other2', 'Other3']:
            if pd.notnull(row[col]) and row[col] != '':
                values.append(row[col])
        # Set dictionary entry
        result[row['Client Name']] = values
    return result

In [ ]:
def fetch_client_names():
    try:
        gc, _ = authenticate_gdrive()
        #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
        #gc = gspread.authorize(creds)
        spreadsheet = gc.open('Additional Data')
        sheet = spreadsheet.worksheet('Client Names Sheet')
        # Get all records from the sheet
        data = sheet.get_all_records()
        df = pd.DataFrame(data)
        df = df[~df['Other1'].isnull() & (df['Other1'].str.strip() != '')]  #ignore those for which no alternative name is listed
        ClientNames_dict = clients_df_to_custom_dict(df)
        return ClientNames_dict # return as dictionary 
    
    except Exception as e:
        logging.error(f"Failed to fetch Client Names Sheet: {e}")
        raise   

In [ ]:
def standardize_client_names(df, df_name):

    ClientNames_dict = fetch_client_names()
    Nonstrd_ClientNames_list = [item for value in ClientNames_dict.values() for item in (value if isinstance(value, (list, tuple)) else [value])]

    Nonstrd_ClientNames_list = [clean_string(i) for i in Nonstrd_ClientNames_list]
    def check_match(text):
        if pd.isna(text):  # Handle NaN/None values
            return False
        cleaned_text = clean_string(text)
        return cleaned_text in Nonstrd_ClientNames_list
    
    orginal_colname = df.columns[0]
    orginal_cols = list(df.columns)

    # Standardize column
    df = df.rename(columns={df.columns[0]: 'temp col'})
    # Add a new column 'Match' to the DataFrame
    df['Match'] = df['temp col'].apply(check_match)

    value_map = {}
    for key, values in ClientNames_dict.items():
        for value in values:
            value_map[clean_string(value)] = key
    
    # Function to apply to each row
    def get_standard_value(row):
        if row['Match']:  # If Match is True
            cleaned_col = clean_string(row['temp col'])
            return value_map.get(cleaned_col, row['temp col'])  # Return key if found, else original
        return row['temp col']  # Return original if Match is False
    
    # Add 'standard' column to DataFrame
    df['standard'] = df.apply(get_standard_value, axis=1)
    df = df.drop(columns=['temp col'])
    df = df.rename(columns={'standard': orginal_colname})
    df = df.reindex(columns=orginal_cols) # reorder
    print('Client Names standardized for {}'.format(df_name))
    
    return df

## determine individuals repsonsible for reporting 
determine which Kindle EMployees are (would be) on leave on day of reporting

In [ ]:
def fetch_leave(reporting_date=ReportingDate):
    reporting_date = pd.Timestamp(reporting_date)
    try:
        gc, _ = authenticate_gdrive()
        #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
        #gc = gspread.authorize(creds)
        spreadsheet = gc.open('Additional Data')
        sheet = spreadsheet.worksheet('Leave Sheet')
        data = pd.DataFrame(sheet.get_all_records())

        data['Leave - Start Date'] = pd.to_datetime(data['Leave - Start Date'], format="%d/%m/%Y", errors='coerce')
        data['Leave - End Date'] = pd.to_datetime(data['Leave - End Date'], format="%d/%m/%Y", errors='coerce')
        data['On Leave'] = False

        for index, row in data.iterrows():
            start = row['Leave - Start Date']
            end = row['Leave - End Date']

            if pd.notnull(start) and start < reporting_date <= end:
                data.at[index, 'On Leave'] = True
            elif pd.isnull(start) and pd.notnull(end) and end > reporting_date:
                data.at[index, 'On Leave'] = True

        return data
    except Exception as e:
        logging.error(f"Failed to fetch Leave Sheet: {e}")
        raise

In [ ]:
def fetch_kindle_employees():
    try:
        gc, _ = authenticate_gdrive()
        #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
        #gc = gspread.authorize(creds)
        spreadsheet = gc.open('Additional Data')
        sheet = spreadsheet.worksheet('Person Responsible Sheet')
        data = sheet.get_all_records()
        data = pd.DataFrame(data)
        return data
    except Exception as e:
        logging.error(f"Failed to fetch Person Responsible Sheet: {e}")
        raise

In [ ]:
def determine_person_responsible(reporting_date=ReportingDate):
    reporting_date = pd.Timestamp(reporting_date)
    Employee_data = fetch_kindle_employees()
    LeaveStatus_data = fetch_leave(reporting_date)

    def determine_devs_turn(row):
        rotation_freq = row['Rotation Frequency (Weeks)']
        if pd.isna(rotation_freq) or rotation_freq == '':
        # Return Developer 1's name if valid, otherwise return empty string
            dev1 = row['Developer 1']
            if dev1 and dev1 != '?' and not pd.isna(dev1):
                return dev1.strip()
            return ''
    
        # Convert rotation_freq to an integer
        try:
            rotation_freq = int(rotation_freq)
        except (ValueError, TypeError):
            return ''
    
        if rotation_freq <= 0:
            return ''

        # Determine which developer columns to consider (Developer 1 to Developer n)
        developer_columns = ['Developer 1', 'Developer 2', 'Developer 3', 'Developer 4'][:rotation_freq]
    
        # Get the list of developers from the relevant columns
        developers = []
        for col in developer_columns:
            dev = row[col]
            if dev and dev != '?' and not pd.isna(dev):
            # Handle multiple developers in the same column (e.g., "Joel;Brendan")
                developers.extend([d.strip() for d in dev.split(';') if d.strip()])
    
        if not developers:
            return ''
    
        # Ensure current_week_number is an integer
        if not isinstance(WeekNumber, int):
            raise ValueError(f"current_week_number must be an integer, got {type(WeekNumber)}: {WeekNumber}")
    
        # Calculate the developer index
        developer_index = (WeekNumber % rotation_freq) % len(developers)
    
        # Verify developer_index is an integer
        if not isinstance(developer_index, int):
            raise ValueError(f"developer_index is not an integer, got {type(developer_index)}: {developer_index}")
    
        return developers[developer_index]

    Employee_data['Developer'] = Employee_data.apply(determine_devs_turn, axis=1)

    on_leave_list = LeaveStatus_data.loc[LeaveStatus_data['On Leave'] == True, 'Kindle Employee'].dropna().tolist()

    def assign_functionalerror_reporter(row):
        dev = row['Developer']
        dev2 = row['Developer 2']
        dev3 = row['Developer 3']
        dev4 = row['Developer 4']
        emergency = row['In Case of Emergency']
    
        dev_on_leave = pd.notna(dev) and dev in on_leave_list
        dev2_on_leave = pd.notna(dev2) and dev2 in on_leave_list
        dev3_on_leave = pd.notna(dev3) and dev3 in on_leave_list
        dev4_on_leave = pd.notna(dev4) and dev4 in on_leave_list
    
        if pd.notna(dev) and not dev_on_leave:
            return dev
        elif pd.notna(dev2) and not dev2_on_leave:
            return dev2
        elif pd.notna(dev3) and not dev3_on_leave:
            return dev3
        elif pd.notna(dev4) and not dev4_on_leave:
            return dev4
        elif pd.notna(emergency):
            return emergency
        else:
            return None  # or raise an error if no responsible person can be assigned
        
    Employee_data['Functional Errors Reporter'] = Employee_data.apply(assign_functionalerror_reporter, axis=1)


    def assign_clientmovement_reporter(row):
        if row['Designated Reporter on Client Movement'] == 'Developer':
            return assign_functionalerror_reporter(row)

        if row['Designated Reporter on Client Movement'] == 'Business Analyst':
            ba1 = row['Business Analyst 1']
            ba2 = row['Business Analyst 2']
            emergency = row['In Case of Emergency']

            ba1_on_leave = pd.notna(ba1) and ba1 in on_leave_list
            ba2_on_leave = pd.notna(ba2) and ba2 in on_leave_list

            if pd.notna(ba1) and not ba1_on_leave:
                return ba1
            elif pd.notna(ba2) and not ba2_on_leave:
                return ba2
            elif pd.notna(emergency):
                return emergency
            else:
                return None  # or raise an error if no responsible person can be assigned

        return None  # If "Designated Reporter on Client Movement" is neither BA nor Developer

    
    Employee_data['Client Movement Reporter'] = Employee_data.apply(assign_clientmovement_reporter, axis=1)

    PersonsResponsible_data = Employee_data.drop(['Designated Reporter on Client Movement', 'Business Analyst 1', 'Business Analyst 2', 'Key Resource',
       'Developer 1', 'Developer 2', 'Developer 3', 'Developer 4',
       'Rotation Frequency (Weeks)', 'In Case of Emergency', 'Developer'], axis=1)
    
    PersonsResponsible_data = standardize_client_names(PersonsResponsible_data, 'PersonsResponsible_data')
    
    return PersonsResponsible_data

#### apply to Client Movement and Current Weeks' Functional Errors

In [ ]:
def fetch_current_errors():
    try:
        gc, _ = authenticate_gdrive()
        #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
        #gc = gspread.authorize(creds)
        spreadsheet = gc.open('ERROR REPORT Current Week')
        current_sheet = spreadsheet.worksheet(CurrentSheetName)
        data = current_sheet.get_all_records()
        return pd.DataFrame(data)
    except Exception as e:
        logging.error(f"Failed to fetch Sheet: {e}")
        raise

In [ ]:
def determine_error_count():
    pd.set_option('future.no_silent_downcasting', True)
    
    Current_Error_data = fetch_current_errors()
    
    Current_Error_data['Client'] = Current_Error_data['Client'].replace('', np.nan)
    Current_Error_data['Period'] = Current_Error_data['Period'].replace('', np.nan)
    Current_Error_data['Client'] = Current_Error_data['Client'].ffill().bfill()
    Current_Error_data['Period'] = Current_Error_data['Period'].ffill().bfill()
    Current_Error_data = Current_Error_data.infer_objects(copy=False)
    Current_Error_data['Functional Error Y/N'] = Current_Error_data['Functional Error Y/N'].astype(str).str.strip().str.upper()

    Current_Error_grouped = Current_Error_data.groupby(['Client', 'Period', 'Functional Error Y/N'], as_index=False)['No of times error occurred'].sum()
    Current_Error_grouped.rename(columns={'No of times error occurred': 'Total Errors'}, inplace=True)

    # Step 2: Rename column to remove spaces/slashes
    Current_Error_grouped.rename(columns={'Functional Error Y/N': 'FunctionalError_Y/N'}, inplace=True)

    # Step 3: Pivot Y/N values into columns
    Current_ErrorCount_data = Current_Error_grouped.pivot_table(
        index=['Client', 'Period'],
        columns='FunctionalError_Y/N',
        values='Total Errors',
        aggfunc='sum',
        fill_value=0
        ).reset_index()
    Current_ErrorCount_data = Current_ErrorCount_data.infer_objects(copy=False)
    # Flatten the column index created by the pivot (required!)
    
    Current_ErrorCount_data.columns.name = None  # Remove the "FunctionalError_Y/N" header
    Current_ErrorCount_data = Current_ErrorCount_data.drop(columns=[''])

    # Step 4: Rename pivoted columns for clarity
    Current_ErrorCount_data.rename(columns={'Y': 'FunctionalErrors', 'N': 'NonFunctionalErrors'}, inplace=True)

    Current_ErrorCount_data = standardize_client_names(Current_ErrorCount_data, 'Current_ErrorCount_data')

    return Current_ErrorCount_data

In [ ]:
def fetch_client_movement():
    try:
        gc, _ = authenticate_gdrive()
        #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
        #gc = gspread.authorize(creds)
        spreadsheet = gc.open('Weekly Movement')
        sheet = spreadsheet.worksheet('Export')
        
        data = sheet.get_all_records()
        
       
        ClientMovement_data = pd.DataFrame(data[1:], columns=data[0])

        # Remove all rows after the row where 'Tenant Name' has 'Total'
        if 'Tenant Name' in ClientMovement_data.columns:
            total_index = ClientMovement_data[ClientMovement_data['Tenant Name'].str.contains('Total', na=False)].index
            if not total_index.empty:
                ClientMovement_data = ClientMovement_data.loc[:total_index[0]-1]

        ClientMovement_data = standardize_client_names(ClientMovement_data, 'ClientMovement_data')

        return ClientMovement_data

    except Exception as e:
        logging.error(f"Failed to fetch Client Movement: {e}")
        raise


In [ ]:
PersonsResponsible_data = determine_person_responsible()
BAResponsible_data = PersonsResponsible_data[['Clients','Client Movement Reporter']].rename(columns={'Clients':'Tenant Name'})
DEVResponsible_data = PersonsResponsible_data[['Clients','Functional Errors Reporter']].rename(columns={'Clients':'Company'})

2025-05-21 15:44:23,224 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:44:27,234 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:44:30,401 - INFO - file_cache is only supported with oauth2client<4.0.0
Client Names standardized for PersonsResponsible_data

### AttributeError                            
Traceback (most recent call last)
Cell In[22], line 1
----> 1 PersonsResponsible_data = determine_person_responsible()
      2 BAResponsible_data = PersonsResponsible_data[['Clients','Client Movement Reporter']].rename(columns={'Clients':'Tenant Name'})
      3 DEVResponsible_data = PersonsResponsible_data[['Clients','Functional Errors Reporter']].rename(columns={'Clients':'Company'})

Cell In[18], line 3, in determine_person_responsible(reporting_date)
      1 def determine_person_responsible(reporting_date=ReportingDate):
      2     reporting_date = pd.Timestamp(reporting_date)
----> 3     Employee_data = fetch_kindle_employees()
      4     LeaveStatus_data = fetch_leave(reporting_date)
      6     def determine_devs_turn(row):

Cell In[17], line 3, in fetch_kindle_employees()
      1 def fetch_kindle_employees():
      2     try:
----> 3         gc, _ = authenticate_gdrive()
      4         #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
      5         #gc = gspread.authorize(creds)
      6         spreadsheet = gc.open('Additional Data')

Cell In[11], line 42, in authenticate_gdrive()
     40 SCOPES = ['https://www.googleapis.com/auth/drive']
     41 try:
---> 42     creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=SCOPES)
     43     gc = gspread.authorize(creds)
     44     drive_service = build('drive', 'v3', credentials=creds)

AttributeError: type object 'Credentials' has no attribute 'from_service_account_file'

#### 
from google.oauth2.credentials import Credentials <strong> as UserCredentials </strong>

since conflicts with 

from google.oauth2.service_account import Credentials 

defined earlier

##### Client Movement

In [ ]:
ClientMovement_data = fetch_client_movement()
Onlines_data = pd.merge(ClientMovement_data, BAResponsible_data, on='Tenant Name', how='left')
Onlines_data.loc[Onlines_data['Difference'] <= 0, 'Client Movement Reporter'] = np.nan

try:
    gc, _ = authenticate_gdrive()
    #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
    #gc = gspread.authorize(creds)
    spreadsheet = gc.open('Persons Responsible')

    worksheet_title = 'Onlines'
    try:
        sheet = spreadsheet.worksheet(worksheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=worksheet_title, rows=100, cols=26)
        logging.info(f"Created new worksheet: {worksheet_title}")

    data = [Onlines_data.columns.tolist()] + Onlines_data.astype(object).where(pd.notnull(Onlines_data), None).values.tolist()
        
    # Clear the sheet and write the data
    execute_with_backoff(sheet.clear)
    execute_with_backoff(sheet.update, data, 'A1', value_input_option='RAW')

    # Set the width of the first column (column A)
    set_column_width(sheet, 'A', 250)  
    set_column_width(sheet, 'F', 100) 
    set_column_width(sheet, 'G', 150) 

    # Format the data as a table
    num_rows = len(data)
    num_cols = len(data[0])
    range_notation = f'A1:{chr(65 + num_cols - 1)}{num_rows}'

    # Set header row height using batch update
    spreadsheet.batch_update({
            "requests": [
                {
                    "updateDimensionProperties": {
                        "range": {
                            "sheetId": sheet.id,
                            "dimension": "ROWS",
                            "startIndex": 0,  # Row 1 (0-based index)
                            "endIndex": 1     # Only the first row
                        },
                        "properties": {
                            "pixelSize": 30  # Set height to 30 pixels
                        },
                        "fields": "pixelSize"
                    }
                }
            ]
        })

    # Bold headers and format
    format_cell_range(
            sheet,
            f'A1:{chr(65 + num_cols - 1)}1',
            CellFormat(
                textFormat=TextFormat(bold=True),
                horizontalAlignment='CENTER',
                verticalAlignment='MIDDLE',
                backgroundColor=Color(0.9, 0.9, 0.9)  # Light gray background for headers
            )
        )

    # Add borders to create a table effect
    format_cell_range(
            sheet,
            range_notation,
            CellFormat(
                borders=Borders(
                    top=Border('SOLID'),
                    bottom=Border('SOLID'),
                    left=Border('SOLID'),
                    right=Border('SOLID')
                )
            )
        )

    # Add filter buttons to the table
    spreadsheet.batch_update({
            "requests": [
                {
                    "setBasicFilter": {
                        "filter": {
                            "range": {
                                "sheetId": sheet.id,
                                "startRowIndex": 0,  # Start at row 1
                                "endRowIndex": num_rows,  # End at last row
                                "startColumnIndex": 0,  # Start at column A
                                "endColumnIndex": num_cols  # End at last column
                            }
                        }
                    }
                }
            ]
        })

    logging.info(f"Data successfully added to {worksheet_title} worksheet of Persons Responsible workbook")
except Exception as e:
    logging.error(f"Failed to write DataFrame to sheet, apply formatting, or add filter: {e}")
    raise

##### Functional Errors 

In [ ]:
Current_ErrorCount_data = determine_error_count()
Current_ErrorCount_data = Current_ErrorCount_data.rename(columns={'Client': 'Company'}) 
Current_ErrorCount_data = Current_ErrorCount_data.drop(['Period'], axis=1)
FunctionalErrors_data = pd.merge(Current_ErrorCount_data, DEVResponsible_data, on='Company', how='left')
FunctionalErrors_data.loc[FunctionalErrors_data['FunctionalErrors'] > 0, 'Functional Errors Reporter'] = np.nan

try:
    gc, _ = authenticate_gdrive()
    #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
    #gc = gspread.authorize(creds)
    spreadsheet = gc.open('Persons Responsible')

    # Check if worksheet exists, create if it doesn't
    worksheet_title = 'Functional Errors'
    try:
        sheet = spreadsheet.worksheet(worksheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=worksheet_title, rows=100, cols=26)
        logging.info(f"Created new worksheet: {worksheet_title}")

    data = [FunctionalErrors_data.columns.tolist()] + FunctionalErrors_data.astype(object).where(pd.notnull(FunctionalErrors_data), None).values.tolist()
        
    # Clear the sheet and write the data
    execute_with_backoff(sheet.clear)
    execute_with_backoff(sheet.update, data, 'A1', value_input_option='RAW')

        # Set the width of the first column (column A)
    set_column_width(sheet, 'A', 300)
    set_column_width(sheet, 'B', 50)  
    set_column_width(sheet, 'C', 50) 
    set_column_width(sheet, 'D', 150)

        # Format the data as a table
    num_rows = len(data)
    num_cols = len(data[0])
    range_notation = f'A1:{chr(65 + num_cols - 1)}{num_rows}'

        # Set header row height using batch update
    spreadsheet.batch_update({
            "requests": [
                {
                    "updateDimensionProperties": {
                        "range": {
                            "sheetId": sheet.id,
                            "dimension": "ROWS",
                            "startIndex": 0,  # Row 1 (0-based index)
                            "endIndex": 1     # Only the first row
                        },
                        "properties": {
                            "pixelSize": 30  # Set height to 30 pixels
                        },
                        "fields": "pixelSize"
                    }
                }
            ]
        })

        # Bold headers and format
    format_cell_range(
            sheet,
            f'A1:{chr(65 + num_cols - 1)}1',
            CellFormat(
                textFormat=TextFormat(bold=True),
                horizontalAlignment='CENTER',
                verticalAlignment='MIDDLE',
                backgroundColor=Color(0.9, 0.9, 0.9)  # Light gray background for headers
            )
        )

        # Add borders to create a table effect
    format_cell_range(
            sheet,
            range_notation,
            CellFormat(
                borders=Borders(
                    top=Border('SOLID'),
                    bottom=Border('SOLID'),
                    left=Border('SOLID'),
                    right=Border('SOLID')
                )
            )
        )


    # Add filter buttons to the table
    spreadsheet.batch_update({
            "requests": [
                {
                    "setBasicFilter": {
                        "filter": {
                            "range": {
                                "sheetId": sheet.id,
                                "startRowIndex": 0,  # Start at row 1
                                "endRowIndex": num_rows,  # End at last row
                                "startColumnIndex": 0,  # Start at column A
                                "endColumnIndex": num_cols  # End at last column
                            }
                        }
                    }
                }
            ]
        })

    logging.info(f"Data sucessfully added to {worksheet_title} worksheet of Persons Responsible workbook")
except Exception as e:
    logging.error(f"Failed to write DataFrame to sheet, apply formatting, or add filter: {e}")
    raise

2025-05-21 15:45:05,222 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:45:10,135 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:45:13,936 - INFO - file_cache is only supported with oauth2client<4.0.0
Client Names standardized for Current_ErrorCount_data
2025-05-21 15:45:22,686 - INFO - Data sucessfully added to Functional Errors worksheet of Persons Responsible workbook

# MERGE PREVIOUS WITH NEWEST RESULTS

In [ ]:
def fetch_previous(YearWeek=YearWeek): 
    db_path = os.path.join(background_data_dir, 'previous_errors_n_reponse_times_data.db')
    engine = create_engine(f'sqlite:///{db_path}')
    query = f"""
        SELECT * FROM PreviousErrorData
        WHERE CAST(SUBSTR(Period, 1, 6) AS INTEGER) < {YearWeek}
            """
    Previous_Errors_n_ResponseTimes_data = pd.read_sql(query, engine)
    Previous_Errors_n_ResponseTimes_data = standardize_client_names(Previous_Errors_n_ResponseTimes_data,'Previous_Errors_n_ResponseTimes_data')
    
    return Previous_Errors_n_ResponseTimes_data

In [ ]:
def fetch_response_times(data_folder = data_supplied_dir): # change to allow user input 
    #date_string = f"{RunDate .day:02d} {RunDate .month:02d} {RunDate .year}"
    #target_filename = f"KIT 3 Online Reporting Data {date_string}.xlsx"

    files = os.listdir(data_folder)
    matched_file = None
    for file in files:
        if clean_string(file) == clean_string(Designated_Filename):
            matched_file = file
            break

    if not matched_file:
        available_files = ", ".join(files)
        raise FileNotFoundError(f"File '{Designated_Filename}' not found. Available files: {available_files}")

    file_path = os.path.join(data_folder, matched_file)

    try:
        xl = pd.ExcelFile(file_path)
    except Exception as e:
        raise Exception(f"File found but couldn't be accessed: {str(e)}")

    sheet_name = "This week response times"
    if sheet_name not in xl.sheet_names:
        sheet_name = xl.sheet_names[1]  # Default to last sheet

    try:
        data = xl.parse(sheet_name, header=0)
    except Exception as e:
        raise Exception(f"No valid data found in the '{sheet_name}' sheet: {str(e)}")

    Current_ResponseTimes_data  = standardize_client_names(data, 'Current_ResponseTimes_data')
    Current_ResponseTimes_data = Current_ResponseTimes_data.drop(['FunctionalError', 'NonFunctionalError'], axis=1)
    
    #Current_ResponseTimes_data.to_excel("ResponseTimesQuery.xlsx", index=False)

    return Current_ResponseTimes_data

In [ ]:
def concat_errors_n_responsetimes():
    """
    Concatenate current response times and error counts, removing total row and duplicates,
    and append new records to the database.
    """
    # Fetch data
    Current_ResponseTimes_data = fetch_response_times()
    Current_ResponseTimes_data = Current_ResponseTimes_data[~Current_ResponseTimes_data['Period'].isna()]
    Current_ResponseTimes_data['Period'] = Current_ResponseTimes_data['Period'].astype(str)
    Current_ResponseTimes_data['Company'] = Current_ResponseTimes_data['Company'].astype(str)

    Current_ErrorCount_data = determine_error_count()
    Current_ErrorCount_data = Current_ErrorCount_data.rename(columns={'Client': 'Company'})
    Current_ErrorCount_data = Current_ErrorCount_data.drop(['Period'], axis=1)
    Current_ErrorCount_data['Company'] = Current_ErrorCount_data['Company'].astype(str)

    # Merge data
    Current_Errors_n_ResponseTimes_data = pd.merge(
        Current_ResponseTimes_data, Current_ErrorCount_data, on=['Company'], how='left'
    )
    
    # Convert error columns to Int64
    columns_to_convert = ['FunctionalErrors', 'NonFunctionalErrors']
    for col in columns_to_convert:
        Current_Errors_n_ResponseTimes_data[col] = pd.to_numeric(
            Current_Errors_n_ResponseTimes_data[col], errors='coerce'
        ).astype('Int64')

    # Remove duplicates
    duplicates = Current_Errors_n_ResponseTimes_data[
        Current_Errors_n_ResponseTimes_data.duplicated(subset=['Period', 'Company'], keep=False)
    ]
    if not duplicates.empty:
        print("Warning: Removing duplicates based on Period and Company:")
        print(duplicates[['Period', 'Company']])
        Current_Errors_n_ResponseTimes_data = Current_Errors_n_ResponseTimes_data.drop_duplicates(
            subset=['Period', 'Company']
        )

    # Database setup
    db_path = os.path.join(background_data_dir, 'previous_errors_n_reponse_times_data.db')
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    engine = create_engine(f'sqlite:///{db_path}', echo=True)

    # Verify period
    try:
        with engine.connect() as conn:
            inspector = inspect(engine)
            if not inspector.has_table('PreviousErrorData'):
                print("Warning: PreviousErrorData table does not exist. Creating it.")
                Current_Errors_n_ResponseTimes_data.to_sql(
                    'PreviousErrorData', con=engine, if_exists='replace', index=False
                )
            else:
                last_entry = pd.read_sql(
                    "SELECT Period FROM PreviousErrorData ORDER BY Period DESC LIMIT 1", conn
                )
                if last_entry.empty:
                    print("Warning: PreviousErrorData table is empty. Cannot verify Period.")
                else:
                    last_period = last_entry['Period'].iloc[0]
                    last_period_prefix = int(last_period[:6])
                    new_period_prefix = int(Current_ResponseTimes_data['Period'].iloc[0][:6])
                    if new_period_prefix == last_period_prefix + 1:
                        print(
                            f"Verification successful: New Period ({new_period_prefix}) is one week after last Period ({last_period_prefix})."
                        )
                    else:
                        print(
                            f"Verification failed: New Period ({new_period_prefix}) is not one more than last Period ({last_period_prefix})."
                        )
    except Exception as e:
        print(f"Error during verification: {e}")
        with open('error_log.txt', 'a') as f:
            f.write(f"Error during verification: {e}\n")

    # Append new records
    try:
        with engine.connect() as conn:
            existing_records = pd.read_sql(
                "SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods",
                conn,
                params={'periods': tuple(Current_Errors_n_ResponseTimes_data['Period'].unique())}
            )

            # Ensure consistent data types
            for df in [Current_Errors_n_ResponseTimes_data, existing_records]:
                df['Period'] = df['Period'].astype(str)
                df['Company'] = df['Company'].astype(str)

            new_records = Current_Errors_n_ResponseTimes_data.merge(
                existing_records,
                on=['Period', 'Company'],
                how='left',
                indicator=True
            ).query('_merge == "left_only"').drop(columns='_merge')

            if new_records.empty:
                print("No new records to append; all records already exist in PreviousErrorData.")
            else:
                new_records.to_sql('PreviousErrorData', con=engine, if_exists='append', index=False)
                print(f"Successfully appended {len(new_records)} new records to PreviousErrorData table.")
                print("Appended records:")
                print(new_records[['Period', 'Company']])
    except Exception as e:
        print(f"Error checking/appending data: {e}")
        with open('error_log.txt', 'a') as f:
            f.write(f"Error checking/appending data: {e}\n")

    # Fetch previous data and concatenate
    Previous_Errors_n_ResponseTimes_data = fetch_previous()
    Updated_Errors_n_ResponseTimes_data = pd.concat(
        [Previous_Errors_n_ResponseTimes_data, Current_Errors_n_ResponseTimes_data],
        ignore_index=True
    )
    Updated_Errors_n_ResponseTimes_data = Updated_Errors_n_ResponseTimes_data.drop_duplicates(
        subset=['Period', 'Company'], keep='last'
    )

    return Updated_Errors_n_ResponseTimes_data

#### write to report

In [ ]:
Updated_Errors_n_ResponseTimes_data = concat_errors_n_responsetimes()
Updated_Errors_n_ResponseTimes_data = Updated_Errors_n_ResponseTimes_data.where(pd.notnull(Updated_Errors_n_ResponseTimes_data), None)
ClientNames_dict = fetch_client_names()
Nonstrd_ClientNames_list = [item for value in ClientNames_dict.values() for item in (value if isinstance(value, (list, tuple)) else [value])]
if not Updated_Errors_n_ResponseTimes_data['Company'].isin(Nonstrd_ClientNames_list).any():
    print("No matches found.")
else:
    print("There is at least one match.")

2025-05-21 15:46:01,159 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 15:46:01,159 - INFO - BEGIN (implicit)
2025-05-21 15:46:01,167 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods")
2025-05-21 15:46:01,167 - INFO - PRAGMA main.table_info("SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods")
2025-05-21 15:46:01,173 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 15:46:01,173 - INFO - [raw sql] ()
2025-05-21 15:46:01,174 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods")
2025-05-21 15:46:01,174 - INFO - PRAGMA temp.table_info("SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods")
2025-05-21 15:46:01,176 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 15:46:01,176 - INFO - [raw sql] ()
2025-05-21 15:46:01,179 INFO sqlalchemy.engine.Engine SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods
2025-05-21 15:46:01,179 - INFO - SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods
2025-05-21 15:46:01,180 INFO sqlalchemy.engine.Engine [raw sql] {'periods': ('202517 - 24 Apr 2025 - 01 May 2025',)}
2025-05-21 15:46:01,180 - INFO - [raw sql] {'periods': ('202517 - 24 Apr 2025 - 01 May 2025',)}
2025-05-21 15:46:01,182 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-21 15:46:01,182 - INFO - ROLLBACK
2025-05-21 15:46:01,229 - INFO - file_cache is only supported with oauth2client<4.0.0
Error checking/appending data: (sqlite3.OperationalError) near ":periods": syntax error
[SQL: SELECT Period, Company FROM PreviousErrorData WHERE Period IN :periods]
[parameters: {'periods': ('202517 - 24 Apr 2025 - 01 May 2025',)}]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
2025-05-21 15:46:03,902 - INFO - file_cache is only supported with oauth2client<4.0.0
Client Names standardized for Previous_Errors_n_ResponseTimes_data
No matches found.

In [ ]:
try:
    gc, _ = authenticate_gdrive()
    #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
    #gc = gspread.authorize(creds)
    spreadsheet = gc.open('KIT 3 Online Reporting Data')

    # Check if worksheet exists, create if it doesn't
    worksheet_title = 'Data Errors and Reponse Times'
    try:
        sheet = spreadsheet.worksheet(worksheet_title)
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=worksheet_title, rows=100, cols=26)  # Initial size: 100 rows, 26 cols (A-Z)
        logging.info(f"Created new worksheet: {worksheet_title}")

    # Prepare data: Fill NaN values based on column type
    data_filled = Updated_Errors_n_ResponseTimes_data.copy()

    for col in data_filled.columns:
        if data_filled[col].dtype == 'Float64':
            data_filled[col] = data_filled[col].fillna(0)
        else:
            data_filled[col] = data_filled[col].fillna('')

    # Convert DataFrame to list for writing to sheet
    data = [data_filled.columns.tolist()] + data_filled.astype(object).where(pd.notnull(data_filled), None).values.tolist()

    # Clear the sheet and write the data
    execute_with_backoff(sheet.clear)
    execute_with_backoff(sheet.update, data, 'A1', value_input_option='RAW')

    # Format the data as a table
    num_rows = len(data)
    num_cols = len(data[0])
    range_notation = f'A1:{chr(65 + num_cols - 1)}{num_rows}'

    # Set header row height using batch update
    spreadsheet.batch_update({
            "requests": [
                {
                    "updateDimensionProperties": {
                        "range": {
                            "sheetId": sheet.id,
                            "dimension": "ROWS",
                            "startIndex": 0,  # Row 1 (0-based index)
                            "endIndex": 1     # Only the first row
                        },
                        "properties": {
                            "pixelSize": 30  # Set height to 30 pixels
                        },
                        "fields": "pixelSize"
                    }
                }
            ]
        })

    # Bold headers and format
    format_cell_range(
        sheet,
        f'A1:{chr(65 + num_cols - 1)}1',
        CellFormat(
                textFormat=TextFormat(bold=True),
                horizontalAlignment='CENTER',
                verticalAlignment='MIDDLE',
                backgroundColor=Color(0.9, 0.9, 0.9)  # Light gray background for headers
            )
        )

    # Add borders to create a table effect
    format_cell_range(
            sheet,
            range_notation,
            CellFormat(
                borders=Borders(
                    top=Border('SOLID'),
                    bottom=Border('SOLID'),
                    left=Border('SOLID'),
                    right=Border('SOLID')
                )
            )
        )

    # Auto-resize columns for better readability
    spreadsheet.batch_update({
            "requests": [
                {
                    "autoResizeDimensions": {
                        "dimensions": {
                            "sheetId": sheet.id,
                            "dimension": "COLUMNS",
                            "startIndex": 0,
                            "endIndex": num_cols
                        }
                    }
                }
            ]
        })

    # Add filter buttons to the table
    spreadsheet.batch_update({
            "requests": [
                {
                    "setBasicFilter": {
                        "filter": {
                            "range": {
                                "sheetId": sheet.id,
                                "startRowIndex": 0,  # Start at row 1
                                "endRowIndex": num_rows,  # End at last row
                                "startColumnIndex": 0,  # Start at column A
                                "endColumnIndex": num_cols  # End at last column
                            }
                        }
                    }
                }
            ]
        })

    logging.info(f"Overwrote {len(Updated_Errors_n_ResponseTimes_data)} rows in {sheet}, formatted as a table, and added filter buttons")
       
except Exception as e:
    logging.error(f"Failed to write DataFrame to sheet, apply formatting, or add filter: {e}")
    raise

2025-05-21 15:46:20,187 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:46:34,533 - INFO - Overwrote 5031 rows in <Worksheet 'Data Errors and Reponse Times' id:1289794990>, formatted as a table, and added filter buttons

#### create accompanying pivot table

In [ ]:
try:
    gc, _ = authenticate_gdrive()
    #creds = Credentials.from_service_account_file(GOOGLE_SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
    #gc = gspread.authorize(creds)
    spreadsheet = gc.open('KIT 3 Online Reporting Data')

    # Access the data sheet
    data_sheet = spreadsheet.worksheet('Data Errors and Reponse Times')

    # Create or access a new sheet for pivot table, slicers, and chart
    pivot_sheet_title = 'Errors Report'
    try:
        pivot_sheet = spreadsheet.worksheet(pivot_sheet_title)
        execute_with_backoff(pivot_sheet.clear)
    except gspread.exceptions.WorksheetNotFound:
        pivot_sheet = spreadsheet.add_worksheet(title=pivot_sheet_title, rows=100, cols=26)
        logging.info(f"Created new worksheet: {pivot_sheet_title}")

    _ , sheets_service = authenticate_gsheets()
    #sheets_service = build('sheets', 'v4', credentials=creds)
    sheet_id = spreadsheet.id
    pivot_sheet_id = pivot_sheet.id

    # Determine column indices based on your data headers
    headers = data_sheet.row_values(1)  # Assuming headers are in the first row
    company_idx = headers.index('Company') if 'Company' in headers else 0
    functional_error_idx = headers.index('Functional Error Count') if 'Functional Error Count' in headers else 1
    non_functional_error_idx = headers.index('Non Functional Error Count') if 'Non Functional Error Count' in headers else 2
    period_idx = headers.index('Period') if 'Period' in headers else 3

    # Define the pivot table with filter for PeriodName
    pivot_table_request = {
            "requests": [
                {
                    "updateCells": {
                        "rows": [
                            {
                                "values": [
                                    {
                                        "pivotTable": {
                                            "source": {
                                                "sheetId": data_sheet.id,
                                                "startRowIndex": 0,
                                                "startColumnIndex": 0,
                                                "endRowIndex": len(data_sheet.get_all_values()),
                                                "endColumnIndex": len(data_sheet.get_all_values()[0])
                                            },
                                            "rows": [
                                                {
                                                    "sourceColumnOffset": company_idx,
                                                    "showTotals": True,
                                                    "sortOrder": "ASCENDING"
                                                }
                                            ],
                                            "values": [
                                                {
                                                    "sourceColumnOffset": functional_error_idx,
                                                    "summarizeFunction": "SUM",
                                                    "name": "Functional Error Count"
                                                },
                                                {
                                                    "sourceColumnOffset": non_functional_error_idx,
                                                    "summarizeFunction": "SUM",
                                                    "name": "Non Functional Error Count"
                                                }
                                            ],
                                            "criteria": {
                                                str(period_idx): {
                                                    "visibleValues": [PeriodName]
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                        ],
                        "start": {
                            "sheetId": pivot_sheet_id,
                            "rowIndex": 0,
                            "columnIndex": 0
                        },
                        "fields": "*"
                    }
                }
            ]
        }

    # Execute pivot table creation
    sheets_service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=pivot_table_request).execute()
    logging.info("Created pivot table in 'Errors Report' sheet")

except Exception as e:
    logging.error(f"Failed to create pivot table or slicers: {e}")
    raise


2025-05-21 15:46:42,892 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:46:47,133 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:46:50,595 - INFO - Created pivot table in 'Errors Report' sheet

# COPY DATA AND PASTE DATA BETWEEN WORKBOOKS 
copy data from Current Errors and Follow Up sheets and add to report

In [ ]:
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID', '1m49V3mKZiuTatraSSGqo1xHLIhcQRmJZO90S4Y72vcc') # CURRENT
DEST_SPREADSHEET_ID = os.getenv('DEST_SPREADSHEET_ID', '1oc1zL7BdlRFFT68cZV46ctvHi5q-ZosaPk79HvireWI') # KIT 3 ONLINE REPORT

gc, sheets_service = authenticate_gsheets()

# Open source and destination spreadsheets
spreadsheet = execute_with_backoff(gc.open_by_key, SPREADSHEET_ID)
dest_spreadsheet = execute_with_backoff(gc.open_by_key, DEST_SPREADSHEET_ID)

# Cache worksheets to reduce API calls
source_worksheets = execute_with_backoff(spreadsheet.worksheets)
dest_worksheets = execute_with_backoff(dest_spreadsheet.worksheets)

source_worksheet_ids = {sheet.title: sheet.id for sheet in source_worksheets}
dest_worksheet_ids = {sheet.title: sheet.id for sheet in dest_worksheets}

# Define sheet names
source_sheet_name = CurrentSheetName  
dest_sheet_name = 'Current Week Error'

# Copy and rename sheet if needed
if source_sheet_name in source_worksheet_ids and source_sheet_name not in dest_worksheet_ids:
    # Copy the sheet
    copy_response = execute_with_backoff(
        sheets_service.spreadsheets().sheets().copyTo,
        spreadsheetId=SPREADSHEET_ID,
        sheetId=source_worksheet_ids[source_sheet_name],
        body={'destinationSpreadsheetId': DEST_SPREADSHEET_ID}
    )
    copied_sheet_id = copy_response['sheetId']
    logging.info(f"Copied sheet '{source_sheet_name}' to destination.")

    # Delete existing destination sheet if it exists
    if dest_sheet_name in dest_worksheet_ids:
        delete_request = {
            'requests': [{
                'deleteSheet': {
                    'sheetId': dest_worksheet_ids[dest_sheet_name]
                }
            }]
        }
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=DEST_SPREADSHEET_ID,
            body=delete_request
        )
        logging.info(f"Deleted existing sheet '{dest_sheet_name}' from destination.")

    # Rename the copied sheet and ensure it's not hidden
    rename_and_unhide_request = {
    'requests': [{
        'updateSheetProperties': {
            'properties': {
                'sheetId': copied_sheet_id,
                'title': dest_sheet_name,
                'hidden': False
            },
            'fields': 'title,hidden'
            }
        }]
    }

    execute_with_backoff(
    sheets_service.spreadsheets().batchUpdate,
    spreadsheetId=DEST_SPREADSHEET_ID,
    body=rename_and_unhide_request
    )
    logging.info(f"Renamed copied sheet to '{dest_sheet_name}' and ensured it is visible.")


    # Adjust column widths
    resize_columns_request = {
    'requests': [
        {
            'updateDimensionProperties': {
                'range': {
                    'sheetId': copied_sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': 0,  # Column A
                    'endIndex': 1
                },
                'properties': {
                    'pixelSize': 170  # Slightly increased
                },
                'fields': 'pixelSize'
            }
        },
        {
            'updateDimensionProperties': {
                'range': {
                    'sheetId': copied_sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': 1,  # Column B
                    'endIndex': 2
                },
                'properties': {
                    'pixelSize': 120  # Slightly increased
                },
                'fields': 'pixelSize'
            }
        },
        {
            'updateDimensionProperties': {
                'range': {
                    'sheetId': copied_sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': 2,  # Column C
                    'endIndex': 3
                },
                'properties': {
                    'pixelSize': 400  # Significantly increased
                },
                'fields': 'pixelSize'
                }
            }
        ]
    }

    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=DEST_SPREADSHEET_ID,
        body=resize_columns_request
    )
    logging.info("Adjusted column widths for columns A, B (slightly), and C (significantly).")

else:
    logging.info(f"No sheet copied. Either '{source_sheet_name}' doesn't exist in source or '{dest_sheet_name}' already exists in destination.")

2025-05-20 10:06:49,144 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-20 10:06:53,797 - INFO - Copied sheet '24 Apr - 30 Apr' to destination.
2025-05-20 10:06:54,907 - INFO - Deleted existing sheet 'Current Week Error' from destination.
2025-05-20 10:06:55,495 - INFO - Renamed copied sheet to 'Current Week Error' and ensured it is visible.
2025-05-20 10:06:56,035 - INFO - Adjusted column widths for columns A, B (slightly), and C (significantly).

In [ ]:
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID', '13IpNi1rFg8luMX8t_OlhCbAQcEhkZkZUmwFWP3C_8N8') # FOLLOW UP
DEST_SPREADSHEET_ID = os.getenv('DEST_SPREADSHEET_ID', '1oc1zL7BdlRFFT68cZV46ctvHi5q-ZosaPk79HvireWI') # KIT3 REPORT

gc, sheets_service = authenticate_gsheets()

# Open source and destination spreadsheets
spreadsheet = execute_with_backoff(gc.open_by_key, SPREADSHEET_ID)
dest_spreadsheet = execute_with_backoff(gc.open_by_key, DEST_SPREADSHEET_ID)

# Cache worksheets to reduce API calls
source_worksheets = execute_with_backoff(spreadsheet.worksheets)
dest_worksheets = execute_with_backoff(dest_spreadsheet.worksheets)

source_worksheet_ids = {sheet.title: sheet.id for sheet in source_worksheets}
dest_worksheet_ids = {sheet.title: sheet.id for sheet in dest_worksheets}

# Define sheet names
source_sheet_name = PreviousSheetName
dest_sheet_name = 'Feedback on Previous Week Error'

# Copy and rename sheet if needed
if source_sheet_name in source_worksheet_ids and source_sheet_name not in dest_worksheet_ids:
    # Copy the sheet
    copy_response = execute_with_backoff(
        sheets_service.spreadsheets().sheets().copyTo,
        spreadsheetId=SPREADSHEET_ID,
        sheetId=source_worksheet_ids[source_sheet_name],
        body={'destinationSpreadsheetId': DEST_SPREADSHEET_ID}
    )
    copied_sheet_id = copy_response['sheetId']
    logging.info(f"Copied sheet '{source_sheet_name}' to destination.")

    # Delete existing destination sheet if it exists
    if dest_sheet_name in dest_worksheet_ids:
        delete_request = {
            'requests': [{
                'deleteSheet': {
                    'sheetId': dest_worksheet_ids[dest_sheet_name]
                }
            }]
        }
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=DEST_SPREADSHEET_ID,
            body=delete_request
        )
        logging.info(f"Deleted existing sheet '{dest_sheet_name}' from destination.")

    # Rename the copied sheet and ensure it's not hidden
    rename_and_unhide_request = {
    'requests': [{
        'updateSheetProperties': {
            'properties': {
                'sheetId': copied_sheet_id,
                'title': dest_sheet_name,
                'hidden': False
            },
            'fields': 'title,hidden'
            }
        }]
    }

    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=DEST_SPREADSHEET_ID,
        body=rename_and_unhide_request
    )
    logging.info(f"Renamed copied sheet to '{dest_sheet_name}' and ensured it is visible.")


    # Adjust column widths
    resize_columns_request = {
    'requests': [
        {
            'updateDimensionProperties': {
                'range': {
                    'sheetId': copied_sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': 0,  # Column A
                    'endIndex': 1
                },
                'properties': {
                    'pixelSize': 170  # Slightly increased
                },
                'fields': 'pixelSize'
            }
        },
        {
            'updateDimensionProperties': {
                'range': {
                    'sheetId': copied_sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': 1,  # Column B
                    'endIndex': 2
                },
                'properties': {
                    'pixelSize': 120  # Slightly increased
                },
                'fields': 'pixelSize'
            }
        },
        {
            'updateDimensionProperties': {
                'range': {
                    'sheetId': copied_sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': 2,  # Column C
                    'endIndex': 3
                },
                'properties': {
                    'pixelSize': 400  # Significantly increased
                },
                'fields': 'pixelSize'
                }
            }
        ]
    }

    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=DEST_SPREADSHEET_ID,
        body=resize_columns_request
    )
    logging.info("Adjusted column widths for columns A, B (slightly), and C (significantly).")

else:
    logging.info(f"No sheet copied. Either '{source_sheet_name}' doesn't exist in source or '{dest_sheet_name}' already exists in destination.")

2025-05-20 10:06:56,074 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-20 10:07:02,647 - INFO - Copied sheet '17 Apr - 23 Apr' to destination.
2025-05-20 10:07:03,264 - INFO - Deleted existing sheet 'Feedback on Previous Week Error' from destination.
2025-05-20 10:07:03,869 - INFO - Renamed copied sheet to 'Feedback on Previous Week Error' and ensured it is visible.
2025-05-20 10:07:04,503 - INFO - Adjusted column widths for columns A, B (slightly), and C (significantly).

In [ ]:
"""start of alteration"""
designated_file_path = os.path.join(reports_dir, Designated_Filename) # specify using os.path.join for futureproffing with pyw file
#designated_file_path = f'Reports Folder/{Designated_Filename}'


### download file 

In [ ]:
_, drive_service = authenticate_gdrive()

# Spreadsheet ID
SPREADSHEET_ID = '1oc1zL7BdlRFFT68cZV46ctvHi5q-ZosaPk79HvireWI'

# Export Google Sheet as Excel
request = drive_service.files().export_media(
    fileId=SPREADSHEET_ID,
    mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
)


fh = io.FileIO(designated_file_path, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while not done:
    status, done = downloader.next_chunk()


2025-05-20 10:28:29,605 - INFO - file_cache is only supported with oauth2client<4.0.0

### open folder location for manual alternation

In [ ]:
folder_path = os.path.join(script_dir, 'Reports Folder') #r"C:\Users\nicola\Desktop\VisualCode Workspace\Team Meeting Update\Reports Folder"
os.startfile(folder_path)

#file_path = folder_path + '\\' + Designated_Filename
file_path = os.path.join(folder_path, Designated_Filename)
os.startfile(file_path)

### confirm manual alteration
needs troubleshooting - gets stuck in loop

In [ ]:
def ask_user_confirmation():
    root = tk.Tk()
    result = messagebox.askyesno("Confirmation", "Has the file been checked and graph added?")
    root.destroy()  # Destroy the hidden main window
    return result

ask_user_confirmation()
if not ask_user_confirmation():
    print("User did not confirm. Exiting.")
    exit()

# Rest of your code goes here
print("User confirmed. Continuing with the rest of the program.")


In [ ]:
import tkinter as tk
from tkinter import messagebox

def show_confirmation():
    # Create a root window and hide it (needed for messagebox to work)
    root = tk.Tk()
    root.withdraw()

    # Show the Yes/No confirmation popup
    confirmed = messagebox.askyesno("Confirmation", "Has the file been checked and graph added?")
    
    root.destroy()  # Destroy the hidden root window after the dialog

    return confirmed

# Show the confirmation dialog
if show_confirmation():
    print("User confirmed. Continuing with the rest of the code...")
else:
    print("User did not confirm. Exiting or taking alternative action.")


User confirmed. Continuing with the rest of the code...

### email 

In [ ]:

GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.send']
TOKEN_FILE = os.path.join(background_data_dir, 'GmailToken.json') #'Background Data/GmailToken.json' # change to os.path method
CREDENTIALS_FILE = os.path.join(background_data_dir, 'GmailAuth.json') #'Background Data/GmailAuth.json' # change to os.path method

creds = None
if os.path.exists(TOKEN_FILE):
    try:
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, GMAIL_SCOPES)
        print(f"Loaded token, valid: {creds.valid}, expired: {creds.expired}, expiry: {creds.expiry}, refresh_token: {creds.refresh_token}")
    except Exception as e:
        print(f"Error loading token: {e}")
        os.remove(TOKEN_FILE)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        try:
            creds.refresh(Request())
            print("Token refreshed successfully.")
        except RefreshError as e:
            print(f"Refresh failed: {e}")
            print(f"Error details: {e.args}")
            os.remove(TOKEN_FILE)
            creds = None
        except Exception as e:
            print(f"Unexpected refresh error: {e}")
            os.remove(TOKEN_FILE)
            creds = None
    if not creds:
        try:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, GMAIL_SCOPES)
            creds = flow.run_local_server(port=0)
            print("OAuth flow completed.")
        except Exception as e:
            print(f"OAuth flow failed: {e}")
            exit()

    try:
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
        print(f"Token saved to {os.path.abspath(TOKEN_FILE)}")
    except Exception as e:
        print(f"Error saving token: {e}")
        exit()

print("Authentication complete.")


"""end alteration"""

Loaded token, valid: True, expired: False, expiry: 2025-05-20 10:04:46, refresh_token: 1//03Gq_DGcAqaPrCgYIARAAGAMSNwF-L9IrMdWx2R8GaUtcV8ZgX7Bib0l82DVsyRYYHW_WIR-vVhwqcTnbG5-tDhrg1Tu3X7fypls
Authentication complete.

# CREATE NEW SHEETS 
add new column to Follow Up

In [ ]:
# Hardcoded column widths (in pixels) for specific columns (0-based index)
COLUMN_WIDTHS = {
    0: 200,  # Column A
    1: 150,  # Column B
    2: 300,  # Column C
    3: 150,  # Column D
    4: 150, #E
    5: 150, #F
    6: 150, #G
    7: 150, 
    8: 150,
    9: 150
}

CURRENT_SPREADSHEET_ID = os.getenv('CURRENT_SPREADSHEET_ID', '1m49V3mKZiuTatraSSGqo1xHLIhcQRmJZO90S4Y72vcc')  # CURRENT
FOLLOWUP_SPREADSHEET_ID = os.getenv('FOLLOWUP_SPREADSHEET_ID', '13IpNi1rFg8luMX8t_OlhCbAQcEhkZkZUmwFWP3C_8N8')  # FOLLOW UP
TEMPLATE_NAME = 'TEMPLATE - Do not edit'

gc, sheets_service = authenticate_gsheets()

# Open spreadsheets
current_spreadsheet = execute_with_backoff(gc.open_by_key, CURRENT_SPREADSHEET_ID)
template_sheet = current_spreadsheet.get_worksheet_by_id(552319826)
followup_spreadsheet = execute_with_backoff(gc.open_by_key, FOLLOWUP_SPREADSHEET_ID)

# Cache worksheets to reduce API calls
current_worksheets = execute_with_backoff(current_spreadsheet.worksheets)
followup_worksheets = execute_with_backoff(followup_spreadsheet.worksheets)

current_worksheet_ids = {sheet.title: sheet.id for sheet in current_worksheets}
followup_worksheet_ids = {sheet.title: sheet.id for sheet in followup_worksheets}

# Define sheet names 
next_sheet_name = NextSheetName  
followup_sheet_name = CurrentSheetName  

if TEMPLATE_NAME in current_worksheet_ids:
    template_sheet_id = current_worksheet_ids[TEMPLATE_NAME]
    unhide_request = {
        'requests': [{
            'updateSheetProperties': {
                'properties': {
                    'sheetId': template_sheet_id,
                    'hidden': False
                },
                'fields': 'hidden'
            }
        }]
    }
    try:
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=CURRENT_SPREADSHEET_ID,
            body=unhide_request
        )
        logging.info(f"Unhid template sheet '{TEMPLATE_NAME}'.")
    except Exception as e:
        logging.error(f"Error unhiding template sheet '{TEMPLATE_NAME}': {e}")
        raise
else:
    logging.error(f"Template sheet '{TEMPLATE_NAME}' not found.")
    raise Exception(f"Template sheet '{TEMPLATE_NAME}' not found.")

if next_sheet_name in current_worksheet_ids:
    logging.info(f"Worksheet '{next_sheet_name}' already exists. Deleting it.")
    sheet_id = current_worksheet_ids[next_sheet_name]
    delete_request = {
        'requests': [{
            'deleteSheet': {
                'sheetId': sheet_id
            }
        }]
    }
    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=CURRENT_SPREADSHEET_ID,
        body=delete_request
    )
    
    try:   
        del current_worksheet_ids[next_sheet_name]
        time.sleep(1)
        updated_current_worksheets = execute_with_backoff(current_spreadsheet.worksheets)
        updated_current_worksheet_ids = {sheet.title: sheet.id for sheet in updated_current_worksheets}
        
        if next_sheet_name in updated_current_worksheet_ids:
            raise Exception(f"Failed to delete worksheet '{next_sheet_name}'.")
        logging.info(f"Deleted worksheet '{next_sheet_name}'.")
        
    except Exception as e:
        logging.error(f"Error deleting worksheet '{next_sheet_name}': {e}")
        raise

try:
    new_sheet = execute_with_backoff(
        current_spreadsheet.duplicate_sheet,
        source_sheet_id=template_sheet.id,
        new_sheet_name=next_sheet_name)
    
    current_worksheet_ids[next_sheet_name] = new_sheet.id
    
    logging.info(f"Created worksheet '{next_sheet_name}'.")

    # Set column widths for the new sheet
    width_requests = [
        {
            'updateDimensionProperties': {
                'range': {
                    'sheetId': new_sheet.id,
                    'dimension': 'COLUMNS',
                    'startIndex': col_index,
                    'endIndex': col_index + 1
                },
                'properties': {
                    'pixelSize': width
                },
                'fields': 'pixelSize'
            }
        } for col_index, width in COLUMN_WIDTHS.items()
    ]
    if width_requests:
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=CURRENT_SPREADSHEET_ID,
            body={'requests': width_requests}
        )
        logging.info(f"Set column widths for '{next_sheet_name}'.")

    # Update 'Period' column
    values = execute_with_backoff(new_sheet.get_all_values)
    header = values[0]
    try:
        period_col_index = header.index("Period")
        period_col_letter = chr(ord('A') + period_col_index)
    except ValueError:
        raise Exception("Column 'Period' not found.")
    updates = [
        {'range': f"{period_col_letter}{row_idx}", 'values': [[next_sheet_name]]}
        for row_idx, row in enumerate(values[1:], start=2)
        if len(row) > period_col_index and row[period_col_index].strip()
    ]
    if updates:
        execute_with_backoff(new_sheet.batch_update, updates)
        logging.info(f"Updated {len(updates)} 'Period' cells.")

except Exception as e:
    logging.error(f"Error creating worksheet '{next_sheet_name}': {e}")
    raise

if followup_sheet_name in current_worksheet_ids and followup_sheet_name in followup_worksheet_ids:
    logging.info(f"Worksheet '{followup_sheet_name}' already exists. Overwriting it.")
    sheet_id = followup_worksheet_ids[followup_sheet_name]

    # Clear the existing sheet content
    try:
        execute_with_backoff(
            sheets_service.spreadsheets().values().clear,
            spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
            range=f"{followup_sheet_name}!A1:Z",  # Pass range directly
            body={}  # Empty body, as no additional metadata is needed
        )
        logging.info(f"Cleared content of worksheet '{followup_sheet_name}'.")
    except Exception as e:
        logging.error(f"Error clearing worksheet '{followup_sheet_name}': {e}")
        raise

    # Copy data from the source sheet
    try:
        # Get data from the source sheet
        source_data = execute_with_backoff(
            sheets_service.spreadsheets().values().get,
            spreadsheetId=CURRENT_SPREADSHEET_ID,
            range=f"{followup_sheet_name}!A1:Z"
        ).get('values', [])

        # Write data to the existing sheet
        execute_with_backoff(
            sheets_service.spreadsheets().values().update,
            spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
            range=f"{followup_sheet_name}!A1",
            valueInputOption="RAW",
            body={"values": source_data}
        )

        # Update column widths
        width_requests = [
            {
                'updateDimensionProperties': {
                    'range': {
                        'sheetId': sheet_id,
                        'dimension': 'COLUMNS',
                        'startIndex': col_index,
                        'endIndex': col_index + 1
                    },
                    'properties': {
                        'pixelSize': width
                    },
                    'fields': 'pixelSize'
                }
            } for col_index, width in COLUMN_WIDTHS.items()
        ]

        update_request = {'requests': width_requests}
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
            body=update_request
        )

        logging.info(f"Overwrote content and set column widths for sheet '{followup_sheet_name}'.")
    except Exception as e:
        logging.error(f"Error overwriting sheet '{followup_sheet_name}': {e}")
        raise

else:
    # If the sheet doesn't exist, proceed with copying as before
    try:
        copy_response = execute_with_backoff(
            sheets_service.spreadsheets().sheets().copyTo,
            spreadsheetId=CURRENT_SPREADSHEET_ID,
            sheetId=current_worksheet_ids[followup_sheet_name],
            body={'destinationSpreadsheetId': FOLLOWUP_SPREADSHEET_ID}
        )
        
        copied_sheet_id = copy_response['sheetId']
        rename_request = {
            'requests': [{
                'updateSheetProperties': {
                    'properties': {
                        'sheetId': copied_sheet_id,
                        'title': followup_sheet_name
                    },
                    'fields': 'title'
                }
            }]
        }
        # Add column width updates to the rename request
        width_requests = [
            {
                'updateDimensionProperties': {
                    'range': {
                        'sheetId': copied_sheet_id,
                        'dimension': 'COLUMNS',
                        'startIndex': col_index,
                        'endIndex': col_index + 1
                    },
                    'properties': {
                        'pixelSize': width
                    },
                    'fields': 'pixelSize'
                }
            } for col_index, width in COLUMN_WIDTHS.items()
        ]
        rename_request['requests'].extend(width_requests)
        
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
            body=rename_request
        )
        
        followup_worksheet_ids[followup_sheet_name] = copied_sheet_id
        logging.info(f"Copied, renamed, and set column widths for sheet '{followup_sheet_name}'.")
    except Exception as e:
        logging.error(f"Error copying sheet '{followup_sheet_name}': {e}")
        raise

2025-05-21 15:47:13,834 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:47:20,908 - INFO - Unhid template sheet 'TEMPLATE - Do not edit'.
2025-05-21 15:47:20,911 - INFO - Worksheet '01 May - 07 May' already exists. Deleting it.
2025-05-21 15:47:24,535 - INFO - Deleted worksheet '01 May - 07 May'.
2025-05-21 15:47:26,200 - INFO - Created worksheet '01 May - 07 May'.
2025-05-21 15:47:27,230 - INFO - Set column widths for '01 May - 07 May'.
2025-05-21 15:47:29,166 - INFO - Updated 25 'Period' cells.
2025-05-21 15:47:29,171 - INFO - Worksheet '24 Apr - 30 Apr' already exists. Overwriting it.
2025-05-21 15:47:30,349 - INFO - Cleared content of worksheet '24 Apr - 30 Apr'.
2025-05-21 15:47:33,818 - INFO - Overwrote content and set column widths for sheet '24 Apr - 30 Apr'.

In [ ]:
def col_index_to_letter(index):
    result = ''
    index += 1
    while index:
        index, rem = divmod(index - 1, 26)
        result = chr(65 + rem) + result
    return result

In [ ]:
sheet_id = followup_worksheet_ids[followup_sheet_name]

try:
    # Step 1: Get sheet data to find the last column
    spreadsheet = execute_with_backoff(
        sheets_service.spreadsheets().get,
        spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
        ranges=[f"{followup_sheet_name}!A1:ZZ1"],
        includeGridData=True
    )

    sheet_data = next((s for s in spreadsheet['sheets'] if s['properties']['sheetId'] == sheet_id), None)
    if not sheet_data:
        raise Exception(f"Sheet '{followup_sheet_name}' not found in spreadsheet.")

    new_column_index = 10  # Column K
    new_column_letter = 'K'

    # Step 2: Insert a new column
    insert_column_request = {
        'requests': [{
            'insertDimension': {
                'range': {
                    'sheetId': sheet_id,
                    'dimension': 'COLUMNS',
                    'startIndex': new_column_index,
                    'endIndex': new_column_index + 1
                },
                'inheritFromBefore': True
            }
        }]
    }
    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
        body=insert_column_request
    )
    logging.info(f"Inserted new column at index {new_column_index} ({new_column_letter}) in '{followup_sheet_name}'.")

    # Step 3: Set the header to "Comments"
    execute_with_backoff(
        sheets_service.spreadsheets().values().update,
        spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
        range=f"{followup_sheet_name}!{new_column_letter}1",
        valueInputOption='RAW',
        body={'values': [['Comments']]}
    )
    logging.info(f"Set header 'Comments' in column {new_column_letter} of '{followup_sheet_name}'.")

    # Step 4: Get formatting from column F (entire column)
    formatting_response = execute_with_backoff(
        sheets_service.spreadsheets().get,
        spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
        ranges=[f"{followup_sheet_name}!F:F"],
        fields='sheets.data.rowData.values.userEnteredFormat'
    )

    format_data = formatting_response['sheets'][0]['data'][0].get('rowData', [])
    if not format_data:
        logging.warning(f"No formatting data found for column F in '{followup_sheet_name}'. Applying default formatting.")
        column_format = {}
    else:
        column_format = format_data[0]['values'][0].get('userEnteredFormat', {}) if format_data[0].get('values') else {}

    formatting_requests = []
    # Copy formatting for each row in column F
    for row_index, row in enumerate(format_data):
        cell = row.get('values', [])
        row_format = cell[0].get('userEnteredFormat', column_format) if cell else column_format

        # Prepare the request to copy complete formatting including borders
        formatting_requests.append({
            'repeatCell': {
                'range': {
                    'sheetId': sheet_id,
                    'startRowIndex': row_index,
                    'endRowIndex': row_index + 1,
                    'startColumnIndex': new_column_index,
                    'endColumnIndex': new_column_index + 1
                },
                'cell': {
                    'userEnteredFormat': row_format
                },
                'fields': 'userEnteredFormat(backgroundColor,textFormat,borders,padding,horizontalAlignment,verticalAlignment,wrapStrategy,numberFormat)'
            }
        })

    # Apply all formatting requests in one batch
    if formatting_requests:
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
            body={'requests': formatting_requests}
        )
        logging.info(f"Applied complete formatting including borders from column F to column K in '{followup_sheet_name}'.")
    else:
        logging.warning("No formatting data found in column F.")

except Exception as e:
    logging.error(f"Failed to add Comments column to '{followup_sheet_name}': {e}")
    raise

2025-05-21 15:47:49,858 - INFO - Inserted new column at index 10 (K) in '24 Apr - 30 Apr'.
2025-05-21 15:47:50,912 - INFO - Set header 'Comments' in column K of '24 Apr - 30 Apr'.
2025-05-21 15:47:54,406 - INFO - Applied complete formatting including borders from column F to column K in '24 Apr - 30 Apr'.

## hide other sheets

In [ ]:
def filter_hide_requests(worksheet_ids: Dict[str, int], sheets_to_keep_visible: List[str], spreadsheet_id: str) -> List[Dict[str, Any]]:
    gc, sheets_service = authenticate_gsheets() #build('sheets', 'v4', credentials=creds)
    
    try:
        metadata = execute_with_backoff(
            sheets_service.spreadsheets().get,
            spreadsheetId=spreadsheet_id,
            fields='sheets.properties'
        )
        logging.info(f"Metadata retrieved: {metadata.keys()}")
        current_sheets = metadata.get('sheets', [])

        hide_requests = []
        for sheet in current_sheets:
            properties = sheet.get('properties', {})
            title = properties.get('title')
            sheet_id = properties.get('sheetId')
            is_hidden = properties.get('hidden', False)

            if not title or sheet_id is None:
                logging.warning(f"Skipping invalid sheet: {properties}")
                continue

            if title not in sheets_to_keep_visible and not is_hidden:
                hide_requests.append({
                    "updateSheetProperties": {
                        "properties": {
                            "sheetId": sheet_id,
                            "hidden": True
                        },
                        "fields": "hidden"
                    }
                })

        total_sheets = len(current_sheets)
        if len(hide_requests) >= total_sheets:
            logging.warning(f"Prevented hiding all sheets in spreadsheet {spreadsheet_id}.")
            hide_requests = hide_requests[:-1] if hide_requests else []

        return hide_requests

    except Exception as e:
        logging.error(f"Failed to fetch or process current sheet visibility: {e}")
        raise



In [ ]:
current_visible_sheets = [next_sheet_name]
currentworksheet_requests = filter_hide_requests(current_worksheet_ids, current_visible_sheets, CURRENT_SPREADSHEET_ID)

if currentworksheet_requests :
    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=CURRENT_SPREADSHEET_ID,
        body={"requests": currentworksheet_requests }
    )
    logging.info(f"Hid {len(currentworksheet_requests )} sheets in current spreadsheet.")



2025-05-21 15:51:08,167 - INFO - Metadata retrieved: dict_keys(['sheets'])
2025-05-21 15:51:08,735 - INFO - Hid 1 sheets in current spreadsheet.

In [ ]:
followup_visible_sheets = [followup_sheet_name]
followupworksheet_requests = filter_hide_requests(followup_worksheet_ids, followup_visible_sheets, FOLLOWUP_SPREADSHEET_ID)


if followupworksheet_requests :
    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
        body={"requests": followupworksheet_requests }
    )
    logging.info(f"Hid {len(followupworksheet_requests )} sheets in followup spreadsheet.")

2025-05-21 15:51:08,765 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-05-21 15:51:10,584 - INFO - Metadata retrieved: dict_keys(['sheets'])